## Transforming Climata and Housing Data

In [1]:
import pandas as pd
import datetime

## Setting up Weather Dimension

In [2]:
df_weather = pd.read_csv('../webscrapers/weather_stations.csv', parse_dates=['yearmo'], low_memory=False)
df_weather.head(3)

,Long,Lat,Stn_Name,Clim_ID,Prov_or_Ter,Tm,DwTm,D,Tx,DwTx,...,DwP,P%N,S_G,Pd,BS,DwBS,BS%,HDD,CDD,yearmo
0,-123.742,48.935,CHEMAINUS,1011500,BC,3.8,0.0,NaN,10.5,0.0,...,0.0,NaN,0.0,21.0,NaN,NaN,NaN,439.8,0.0,201201
1,-124.133,48.824,COWICHAN LAKE FORESTRY,1012040,BC,2.2,0.0,-1.2,12.5,0.0,...,0.0,114.0,0.0,22.0,NaN,NaN,NaN,489.7,0.0,201201
2,-124.052,48.829,LAKE COWICHAN,1012055,BC,2.1,14.0,-1.4,11.5,9.0,...,9.0,NaN,0.0,15.0,NaN,NaN,NaN,269.5,0.0,201201


In [3]:
#Selecting important features in data
df_weather = df_weather[['Stn_Name','Clim_ID','Prov_or_Ter','Tm','Tx','Tn','P','S','S_G','yearmo']]
df_weather.head(3)

,Stn_Name,Clim_ID,Prov_or_Ter,Tm,Tx,Tn,P,S,S_G,yearmo
0,CHEMAINUS,1011500,BC,3.8,10.5,-5.0,240.6,11.4,0.0,201201
1,COWICHAN LAKE FORESTRY,1012040,BC,2.2,12.5,-7.5,408.1,0.0,0.0,201201
2,LAKE COWICHAN,1012055,BC,2.1,11.5,-8.5,402.2,13.2,0.0,201201


In [4]:
#dropping rows with ####### useless values
df_weather = df_weather[df_weather.P != '######'].copy()
df_weather['P'] = df_weather['P'].astype(float)

In [5]:
#Grouping by location and date to create regional averages
regional_mean=df_weather.groupby(['Prov_or_Ter','yearmo'])[['Tm','Tx','Tn','P','S','S_G']].mean()
regional_mean=regional_mean.reset_index()

column_mappings = {'Tm': 'Temp_Average',
                   'Prov_or_Ter':'location', 
                   'Tx': 'Max_Temp_Average', 
                   'Tn': 'Min_Temp_Average',  
                   'P': 'Precip_Average', 
                   'S': 'Snowfall_Average',
                   'S_G': 'Mean_Snow_Grnd_Last_Day'}

regional_mean.rename(columns=column_mappings, inplace=True)
regional_mean.head()

,location,yearmo,Temp_Average,Max_Temp_Average,Min_Temp_Average,Precip_Average,Snowfall_Average,Mean_Snow_Grnd_Last_Day
0,AB,201201,-7.383402,11.118672,-34.675104,10.042917,10.700000,7.191781
1,AB,201202,-6.802490,7.707025,-25.254357,11.007500,17.420000,11.261364
2,AB,201203,-1.088477,14.159259,-19.236214,19.475732,26.260656,4.768116
3,AB,201204,4.472016,22.780408,-9.363821,36.970248,15.016923,0.388889
4,AB,201205,9.636364,26.234979,-3.077273,48.488136,2.229091,0.044444


## Setting Up Location Dimension

In [6]:
location_mapping = {
    'CA': 'Canada', 'NL': 'Newfoundland and Labrador', 'PE': 'Prince Edward Island',
    'NS': 'Nova Scotia', 'NB': 'New Brunswick','QC': 'Quebec',
    'ON': 'Ontario', 'MB': 'Manitoba', 'SK': 'Saskatchewan',
    'AB': 'Alberta', 'BC': 'British Columbia', 'YT': 'Yukon',
    'NT': 'Northwest Territories','NU': 'Nunavut'
}
location_dimension = pd.DataFrame.from_dict(location_mapping, orient="index", columns=["fullname"]).reset_index(names='location').reset_index(names="location_key")

In [7]:
location_dimension.rename(columns={'location':'province', 'fullname':'province_fullname'}, inplace=True)

display(location_dimension)
location_dimension.to_csv("location_dimension.csv", index=False)

,location_key,province,province_fullname
0,0,CA,Canada
1,1,NL,Newfoundland and Labrador
2,2,PE,Prince Edward Island
3,3,NS,Nova Scotia
4,4,NB,New Brunswick
5,5,QC,Quebec
6,6,ON,Ontario
7,7,MB,Manitoba
8,8,SK,Saskatchewan
9,9,AB,Alberta


## Setting up date dimension

In [8]:
date_dimension = regional_mean.drop_duplicates("yearmo")[["yearmo"]].copy()

date_dimension["date"] = date_dimension.yearmo.apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m'))

date_dimension["year"] = date_dimension.date.dt.year
date_dimension["month"] = date_dimension.date.dt.month
date_dimension["month_alpha"] = date_dimension.date.dt.month_name()
date_dimension = date_dimension.reset_index(names="date_key")

In [9]:
display(date_dimension)
date_dimension.to_csv("date_dimension.csv", index=False)

,date_key,yearmo,date,year,month,month_alpha
0,0,201201,2012-01-01,2012,1,January
1,1,201202,2012-02-01,2012,2,February
2,2,201203,2012-03-01,2012,3,March
3,3,201204,2012-04-01,2012,4,April
4,4,201205,2012-05-01,2012,5,May
...,...,...,...,...,...,...
129,129,202210,2022-10-01,2022,10,October
130,130,202211,2022-11-01,2022,11,November
131,131,202212,2022-12-01,2022,12,December
132,132,202301,2023-01-01,2023,1,January


## Adding Date and Location surrogate keys onto region_dimension

In [10]:
regional_mean.head(3)

,location,yearmo,Temp_Average,Max_Temp_Average,Min_Temp_Average,Precip_Average,Snowfall_Average,Mean_Snow_Grnd_Last_Day
0,AB,201201,-7.383402,11.118672,-34.675104,10.042917,10.700000,7.191781
1,AB,201202,-6.802490,7.707025,-25.254357,11.007500,17.420000,11.261364
2,AB,201203,-1.088477,14.159259,-19.236214,19.475732,26.260656,4.768116


In [11]:
# Adding Location and Date Surrogate Key to region_dimension
regional_mean = regional_mean.merge(location_dimension[["location_key", "province"]], right_on="province", left_on="location", how="left").drop(["location", "province"], axis=1)
regional_mean = regional_mean.merge(date_dimension[["yearmo", "date_key"]], on="yearmo", how="left").drop("yearmo", axis=1)

In [12]:
regional_mean = regional_mean.reset_index(names="region_summ_key")

In [13]:
regional_mean.rename(columns={
        "Max_Temp_Average":"region_max_temp",
        "Min_Temp_Average":"region_min_temp",
        "Temp_Average":"region_mean_temp",
        "Precip_Average":"region_mean_precip",
        "Mean_Snow_Grnd_Last_Day":"mean_snow_grnd_last_day",
        "Snowfall_Average":"region_mean_snow"
}, inplace=True)
display(regional_mean)
regional_mean.to_csv("regional_weather_summaries.csv", index=False)

,region_summ_key,region_mean_temp,region_max_temp,region_min_temp,region_mean_precip,region_mean_snow,mean_snow_grnd_last_day,location_key,date_key
0,0,-7.383402,11.118672,-34.675104,10.042917,10.700000,7.191781,9,0
1,1,-6.802490,7.707025,-25.254357,11.007500,17.420000,11.261364,9,1
2,2,-1.088477,14.159259,-19.236214,19.475732,26.260656,4.768116,9,2
3,3,4.472016,22.780408,-9.363821,36.970248,15.016923,0.388889,9,3
4,4,9.636364,26.234979,-3.077273,48.488136,2.229091,0.044444,9,4
...,...,...,...,...,...,...,...,...,...
1737,1737,-1.275000,15.710714,-18.221429,54.061538,23.450000,8.636364,11,129
1738,1738,-13.167857,2.596429,-35.492857,12.666667,9.950000,19.153846,11,130
1739,1739,-22.457143,-0.689286,-44.425000,19.766667,21.100000,31.153846,11,131
1740,1740,-14.571429,-0.164286,-28.867857,18.116667,12.950000,39.500000,11,132


## Setting Up Housing Dimension

In [19]:
housing = pd.read_csv('../webscrapers/structure_housing_data_22-Mar-23_08-30PM.csv')
housing.head()

,location,date,total,house,land
0,Canada,January 2012,90.8,89.4,93.8
1,Canada,February 2012,91.0,89.6,93.9
2,Canada,March 2012,91.3,89.9,94.1
3,Canada,April 2012,91.5,90.2,94.2
4,Canada,May 2012,91.7,90.4,94.4


In [20]:
#standardizing date format
housing['yearmo'] = pd.to_datetime(housing.date).apply(lambda x: x.strftime("%Y%m"))
housing.head()

,location,date,total,house,land,yearmo
0,Canada,January 2012,90.8,89.4,93.8,201201
1,Canada,February 2012,91.0,89.6,93.9,201202
2,Canada,March 2012,91.3,89.9,94.1,201203
3,Canada,April 2012,91.5,90.2,94.2,201204
4,Canada,May 2012,91.7,90.4,94.4,201205


In [21]:
# Replacing date with date_key
housing = housing.merge(date_dimension[["date_key", "yearmo"]], on="yearmo", how="left").drop(["yearmo", "date"], axis=1)

# Replacing Location with location_key
housing = housing.merge(location_dimension[["location_key", "province_fullname"]], left_on="location", right_on="province_fullname", how="left").drop(["province_fullname", "location"], axis=1)

# Creating Surrogate Keys
housing = housing.reset_index(names="real_estate_key")

In [22]:
#Saving to CSV
housing.rename(columns={
    "total":"total_price_index",
    "house":"house_price_index",
    "land":"land_price_index"
}, inplace=True)
display(housing)
housing.to_csv("real_estate.csv", index=False)

,real_estate_key,total_price_index,house_price_index,land_price_index,date_key,location_key
0,0,90.8,89.4,93.8,0,0
1,1,91.0,89.6,93.9,1,0
2,2,91.3,89.9,94.1,2,0
3,3,91.5,90.2,94.2,3,0
4,4,91.7,90.4,94.4,4,0
...,...,...,...,...,...,...
1469,1469,127.8,127.6,120.4,129,10
1470,1470,127.7,127.5,120.3,130,10
1471,1471,127.6,127.4,120.2,131,10
1472,1472,127.1,126.8,119.9,132,10


## Creating Fact Table

In [36]:
fact_table = pd.merge(regional_mean[["region_summ_key", "location_key", "date_key"]], housing[["real_estate_key", "location_key", "date_key", "total_price_index"]], how="inner", on=["location_key", "date_key"])
fact_table.head()

,region_summ_key,location_key,date_key,real_estate_key,total_price_index
0,0,9,0,1206,93.2
1,1,9,1,1207,93.2
2,2,9,2,1208,93.6
3,3,9,3,1209,93.7
4,4,9,4,1210,93.9


In [37]:
#Calculating Month over Month Percent Change
fact_table = fact_table.merge(date_dimension[["date_key", "yearmo"]], on="date_key")
fact_table = fact_table.merge(location_dimension[["location_key", "province"]], on="location_key")
fact_table = fact_table.sort_values(["province", "yearmo"])

In [40]:
fact_table["pct_change"] = fact_table.groupby("province")["total_price_index"].pct_change()*100

In [45]:
fact_table = fact_table[fact_table['total_price_index'].notna()]
fact_table = fact_table.drop(["province", "yearmo"], axis=1)

fact_table.to_csv("fact_table.csv", index=False)
fact_table.head()

,region_summ_key,location_key,date_key,real_estate_key,total_price_index,pct_change
0,0,9,0,1206,93.2,NaN
1,1,9,1,1207,93.2,0.000000
2,2,9,2,1208,93.6,0.429185
3,3,9,3,1209,93.7,0.106838
4,4,9,4,1210,93.9,0.213447
